In [1]:
import joblib
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import torch
from mclatte.model import (
    train_mclatte, 
    train_semi_skimmed_mclatte, 
    train_skimmed_mclatte, 
    McLatte,
    SemiSkimmedMcLatte,
    SkimmedMcLatte,
)
from rnn.model import (
    train_baseline_rnn,
    BaselineRnn,
)
from scipy.stats import ttest_ind
from synctwin import io_utils
from synctwin.model import (
    train_synctwin,
    SyncTwinPl,
)

## Data Generation

Constants used for generation

In [ ]:
sim_id = '0.25_200'
seed = 509
model_id = ""
M = 5
H = 5
R = 5 
D = 3 
K = 1 
C = 3

In [ ]:
def generate_data(p_0, N):
    base_path_data = f"data/pkpd/{p_0}_{N}-seed-{seed}"
    data_path = base_path_data + "/{}-{}.{}"

    # loading config and data
    io_utils.load_config(
        data_path, "train"
    )
    x_full, t_full, mask_full, _, y_full, _, _, _, _, _ = io_utils.load_tensor(data_path, "train", device='cpu')
    x_full_val, t_full_val, mask_full_val, _, y_full_val, _, _, _, _, _ = io_utils.load_tensor(data_path, "val", device='cpu')

    x = np.concatenate((x_full.cpu().numpy(), x_full_val.cpu().numpy()), axis=1)
    t = np.concatenate((t_full.cpu().numpy(), t_full_val.cpu().numpy()), axis=1)
    mask = np.concatenate((mask_full.cpu().numpy(), mask_full_val.cpu().numpy()), axis=1)
    y = np.concatenate((y_full.cpu().numpy(), y_full_val.cpu().numpy()), axis=1).squeeze()

    X = x.transpose((1, 0, 2))
    N = X.shape[0]
    rand_index = np.random.permutation(N)
    X = X[rand_index]
    M_ = mask.transpose((1, 0, 2))[rand_index]
    Y_pre = y.T[rand_index]
    Y_post = y.T[rand_index]
    A = np.concatenate((np.zeros((N // 4, 1)), np.ones((N // 4, 1)), np.zeros((N // 4, 1)), np.ones((N // 4, 1))), axis=0)[rand_index]
    T = t.transpose((1, 0, 2))[rand_index]

    N_train = round(N * 0.8)
    N_test = round(N * 0.2)
    X_train, X_test = X[:N_train], X[N_train:]
    M_train, M_test = M_[:N_train], M_[N_train:]
    Y_pre_train, Y_pre_test = Y_pre[:N_train], Y_pre[N_train:]
    Y_post_train, Y_post_test = Y_post[:N_train], Y_post[N_train:]
    A_train, A_test = A[:N_train], A[N_train:]
    T_train, T_test = T[:N_train], T[N_train:]

    return (
        (N_train, M, H, R, D, K, C, X_train, M_train, Y_pre_train, Y_post_train, A_train, T_train),
        (N_test, M, H, R, D, K, C, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test),
    )

In [ ]:
# joblib.dump((N_train, M, H, R, D, K, C, X_train, M_train, Y_pre_train, Y_post_train, A_train, T_train), f'data/pkpd/data_{sim_id}.joblib')
# joblib.dump((N_test, M, H, R, D, K, C, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test), f'data/pkpd/test_data_{sim_id}.joblib')

In [ ]:
def na_catcher(func):
    def wrapper_na_catcher(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(e)
            return np.nan
    return wrapper_na_catcher

## Modelling

### McLatte

In [ ]:
def infer_mcespresso(trained_mcespresso, X_test, A_test, T_test, M_test):
    trained_mcespresso.eval()
    return trained_mcespresso(
        torch.from_numpy(X_test).float(),
        torch.from_numpy(A_test).float(),
        torch.from_numpy(T_test).float(),
        torch.from_numpy(M_test).float(),
    )

#### Vanilla

In [ ]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results_pkpd/mclatte_hp_pkpd.csv')).sort_values(by='valid_loss').iloc[0])
mclatte_config = {
    'encoder_class': 'lstm',
    'decoder_class': 'lstm',
    'hidden_dim': 64,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.001944,
    'gamma': 0.957115,
    'lambda_r': 0.311437,
    'lambda_d': 0.118073,
    'lambda_p': 0.49999,
}

In [ ]:
def test_mclatte(
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    trained_mclatte = train_mclatte(
        mclatte_config,
        X_train,
        M_train,
        Y_pre_train,
        Y_post_train,
        A_train, 
        T_train,
        R,
        M,
        H,
        input_dim=D, 
        treatment_dim=K, 
        test_run=run_idx,
    )
    _, _, y_tilde = infer_mcespresso(
        trained_mclatte, X_test, A_test, T_test, M_test
    )
    
    return torch.nn.functional.l1_loss(
        y_tilde, 
        torch.from_numpy(Y_post_test).float()
    ).item()

#### Semi-Skimmed

In [ ]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results_pkpd/semi_skimmed_mclatte_hp_pkpd.csv')).sort_values(by='valid_loss').iloc[0])
semi_skimmed_mclatte_config = {
    'encoder_class': 'lstm',
    'decoder_class': 'lstm',
    'hidden_dim': 64,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.001944,
    'gamma': 0.957115,
    'lambda_r': 0.311437,
    'lambda_d': 0.118073,
    'lambda_p': 0.49999,
}

In [ ]:
def test_semi_skimmed_mclatte(
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    trained_semi_skimmed_mclatte = train_semi_skimmed_mclatte(
        semi_skimmed_mclatte_config,
        X_train,
        M_train,
        Y_pre_train,
        Y_post_train,
        A_train, 
        T_train,
        R,
        M,
        H,
        input_dim=D, 
        treatment_dim=K, 
        test_run=run_idx,
    )
    _, _, y_tilde = infer_mcespresso(
        trained_semi_skimmed_mclatte, X_test, A_test, T_test, M_test
    )
    
    return torch.nn.functional.l1_loss(
        y_tilde, 
        torch.from_numpy(Y_post_test).float()
    ).item()

#### Skimmed

In [ ]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results_pkpd/skimmed_mclatte_hp_pkpd.csv')).sort_values(by='valid_loss').iloc[0])
skimmed_mclatte_config = {
    'encoder_class': 'lstm',
    'decoder_class': 'lstm',
    'hidden_dim': 64,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.021114,
    'gamma': 0.980614,
    'lambda_r': 0.093878,
    'lambda_p': 0.485204,
}

In [ ]:
def test_skimmed_mclatte(
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    trained_skimmed_mclatte = train_skimmed_mclatte(
        skimmed_mclatte_config,
        X_train,
        M_train,
        Y_pre_train,
        Y_post_train,
        A_train, 
        T_train,
        R,
        M,
        H,
        input_dim=D, 
        treatment_dim=K, 
        test_run=run_idx,
    )
    _, y_tilde = infer_mcespresso(
        trained_skimmed_mclatte, X_test, A_test, T_test, M_test
    )
    
    return torch.nn.functional.l1_loss(
        y_tilde, 
        torch.from_numpy(Y_post_test).float()
    ).item()

### Baseline RNN

In [ ]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results/baseline_rnn_hp_pkpd.csv')).sort_values(by='valid_loss').iloc[0])
rnn_config = {
    'rnn_class': 'gru',
    'hidden_dim': 64,
    'seq_len': 2,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.025182,
    'gamma': 0.543008,
}

In [ ]:
def rnn_predict(trained_rnn, Y_pre, Y_post, return_Y_pred=False):
    """
    Make predictions using results from previous time steps.
    """
    Y = Y_pre
    losses = 0.0
    Y_pred = []
    for i in range(Y_post.shape[1]):
        Y_tilde = trained_rnn(
            torch.from_numpy(Y).float().unsqueeze(2)
        ).squeeze()

        Y = np.concatenate((
            Y[:, 1:], 
            Y_tilde.cpu().detach().numpy()[:, [-1]]
        ), axis=1)
        
        losses += torch.nn.functional.l1_loss(
            Y_tilde[:, -1], 
            torch.from_numpy(Y_post).float()[:, i]
        ).item()
        Y_pred.append(Y_tilde[:, -1])
    if return_Y_pred:
        return torch.stack(Y_pred, 1)
    return losses / Y_post.shape[1]

In [ ]:
def infer_rnn(trained_rnn, Y_pre_test, Y_post_test, return_Y_pred=False):
    trained_rnn.eval()
    return rnn_predict(trained_rnn, Y_pre_test, Y_post_test, return_Y_pred)

In [ ]:
def test_rnn(
    Y_pre_train, 
    Y_pre_test, 
    Y_post_train, 
    Y_post_test, 
    run_idx=0,
):
    trained_rnn = train_baseline_rnn(
        rnn_config,
        Y=np.concatenate((Y_pre_train, Y_post_train), axis=1),
        input_dim=1, 
        test_run=run_idx,
    )

    trained_rnn.eval()
    return rnn_predict(trained_rnn, Y_pre_test, Y_post_test)

### SyncTwin

In [ ]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results/synctwin_hp_pkpd.csv')).sort_values(by='valid_loss').iloc[0])
synctwin_config = {
    'hidden_dim': 128,
    'reg_B': 0.778155,
    'lam_express': 0.658256,
    'lam_recon': 0.086627,
    'lam_prognostic': 0.631468,
    'tau': 0.911613,
    'batch_size': 32,
    'epochs': 100,
    'lr': 0.003222,
    'gamma': 0.572529,
}

In [ ]:
def infer_synctwin(trained_synctwin, N_test, Y_post_test):
    trained_synctwin.eval()
    return trained_synctwin._sync_twin.get_prognostics(
        torch.arange(0, N_test).cpu(),  
        torch.from_numpy(Y_post_test).float().cpu()
    )

In [ ]:
def test_synctwin(
    N_train, 
    N_test, 
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    Y_mask_train = np.all(A_train == 0, axis=1)
    Y_mask_test = np.all(A_test == 0, axis=1)
    Y_control_train = Y_post_train[Y_mask_train]

    trained_synctwin = train_synctwin(
        synctwin_config,
        X=X_train,
        M_=M_train,
        T=T_train,
        Y_batch=Y_post_train,
        Y_control=Y_control_train,
        Y_mask=Y_mask_train, 
        N=N_train,
        D=D,
        n_treated=N_train - Y_control_train.shape[0],
        pre_trt_x_len=R * M,
        test_run=run_idx,
    )

    trained_synctwin.eval()
    _, l1_loss = trained_synctwin(
        torch.from_numpy(X_test).float(),
        torch.from_numpy(T_test).float(),
        torch.from_numpy(M_test).float(),
        torch.arange(0, N_test),
        torch.from_numpy(Y_post_test).float(),
        torch.from_numpy(Y_mask_test).float(),
    )
    return l1_loss.item()

## Test Models

In [ ]:
N_TEST = 5

In [5]:
TEST_CONFIGS = [
    [200, '0.1'],
    [200, '0.25'],
    [200, '0.5'],
    [1000, '0.1'],
    [1000, '0.25'],
    [1000, '0.5'],
]

In [ ]:
for config_idx in range(0):
    config = TEST_CONFIGS[config_idx]
    mclatte_losses = []
    semi_skimmed_mclatte_losses = []
    skimmed_mclatte_losses = []
    rnn_losses = []
    synctwin_losses = []
    for i in range(N_TEST * config_idx + 1, N_TEST * (1 + config_idx) + 1):
        (
            (N_train, M, H, R, D, K, C, X_train, M_train, Y_pre_train, Y_post_train, A_train, T_train),
            (N_test, M, H, R, D, K, C, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test),
        ) = generate_data(config[1], config[0])

        mclatte_losses.append(test_mclatte(
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))
        semi_skimmed_mclatte_losses.append(test_semi_skimmed_mclatte(
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))
        skimmed_mclatte_losses.append(test_skimmed_mclatte(
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))

        rnn_losses.append(test_rnn(
            Y_pre_train,
            Y_pre_test,
            Y_post_train,
            Y_post_test,
            run_idx=i,
        ))

        synctwin_losses.append(test_synctwin(
            N_train, 
            N_test, 
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))
        joblib.dump((
            config, 
            mclatte_losses, 
            semi_skimmed_mclatte_losses, 
            skimmed_mclatte_losses, 
            rnn_losses, 
            synctwin_losses
        ), f'results/test/config_{config_idx}_pkpd.joblib')

### Statistical Testing

In [2]:
LOSS_NAMES = ['McLatte', 'Semi-Skimmed McLatte', 'Skimmed McLatte', 'RNN', 'SyncTwin']

In [9]:
def test_losses(losses):
    t_test_results = pd.DataFrame(columns=LOSS_NAMES, index=LOSS_NAMES)

    for i in range(len(LOSS_NAMES)):
        for j in range(len(LOSS_NAMES)):
            t = ttest_ind(losses[i], losses[j], alternative='less')
            t_test_results[LOSS_NAMES[i]][LOSS_NAMES[j]] = t.pvalue
    return t_test_results

In [12]:
all_losses = [[] for _ in range(len(LOSS_NAMES))]
for config_id in range(len(TEST_CONFIGS)):
    _, *losses = joblib.load(f'results_pkpd/maes/config_{config_id}_pkpd.joblib')
    # print(test_losses(losses))
    for i in range(len(LOSS_NAMES)):
        all_losses[i] += losses[i]

all_results = test_losses(all_losses)
for col_i in LOSS_NAMES:
    print(col_i, end=' ')
    for col_j in LOSS_NAMES:
        print(f'& ' + f'{all_results[col_i][col_j]:.4f}'[1:], end=' ')
    print('\\\\')

McLatte & .5000 & .3377 & .0003 & .0001 & .0000 \\
Semi-Skimmed McLatte & .6623 & .5000 & .0003 & .0001 & .0000 \\
Skimmed McLatte & .9997 & .9997 & .5000 & .0019 & .0000 \\
RNN & .9999 & .9999 & .9981 & .5000 & .1806 \\
SyncTwin & .0000 & .0000 & .0000 & .8194 & .5000 \\


### Plot with trained models

In [ ]:
PLOT_SUB_ID = 1

In [ ]:
def line_model_pred(fig, model, name, infer_model, config_id, plot_sub_id, post_t, y_pre_plot, *infer_args):
    trained_model = model.load_from_checkpoint(os.path.join(os.getcwd(), f'results_pkpd/trained_models/{name}_{config_id + 1}.ckpt'))
    y_tilde = infer_model(trained_model, *infer_args)
    y_pred_plot = [y_pre_plot[-1]] + list(y_tilde.detach().numpy()[plot_sub_id])
    line_pred_model = go.Scatter(x=post_t, y=y_pred_plot, name=f'{name}', line={'dash': 'dash'})
    fig.add_trace(line_pred_model)

In [ ]:
def plot_config_results(config_id):
    config = TEST_CONFIGS[config_id]
    (
        _, (N_test, _, _, _, _, _, _, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test),
    ) = generate_data(config[1], config[0])

    y_pre_plot = Y_pre_test[PLOT_SUB_ID]
    pre_t = list(range(y_pre_plot.shape[0]))

    y_post_plot = [y_pre_plot[-1]] + list(Y_post_test[PLOT_SUB_ID])
    post_t = np.arange(len(y_post_plot)) + y_pre_plot.shape[0] - 1

    fig = go.Figure()
    line_pre_trt = go.Scatter(x=pre_t + list(post_t), y=list(y_pre_plot) + y_post_plot, name='ground truth')
    fig.add_trace(line_pre_trt)

    line_model_pred(fig, SkimmedMcLatte, 'skimmed_mclatte', lambda *args: infer_mcespresso(*args)[1], 
                    config_id, PLOT_SUB_ID, post_t, y_pre_plot, X_test, A_test, T_test, M_test)
    line_model_pred(fig, SemiSkimmedMcLatte, 'semi_skimmed_mclatte', lambda *args: infer_mcespresso(*args)[2], 
                    config_id, PLOT_SUB_ID, post_t, y_pre_plot, X_test, A_test, T_test, M_test)
    line_model_pred(fig, McLatte, 'mclatte', lambda *args: infer_mcespresso(*args)[2], 
                    config_id, PLOT_SUB_ID, post_t, y_pre_plot, X_test, A_test, T_test, M_test)
    line_model_pred(fig, BaselineRnn, 'rnn', lambda *args: infer_rnn(*args, return_Y_pred=True), 
                    config_id, PLOT_SUB_ID, post_t, y_pre_plot, Y_pre_test, Y_post_test)

    print(A_test[PLOT_SUB_ID])
    
    fig.update_layout(
        title='Treatment Outcome', 
        yaxis_title='Outcome', 
        xaxis_title='Time',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig.write_image(f'plots/pkpd/outcome_pred_{config_id}.png')

In [ ]:
for config_id in range(len(TEST_CONFIGS)):
    plot_config_results(config_id)